In [11]:
import mycomm as cmm
import codecs
import pandas as pd
import numpy as np
from string import Template

def fmtOrderBy(v):
	return "" if(v=="") else f"""\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'ASC' THEN f.{v} END ASC,\n\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'DESC' THEN f.{v} END DESC,"""

def fmtUpdateSet(v):
    return "" if(v=="") else f",{v} = @{v}"

def fmtInsertCols(v):
    return "" if(v=="") else f",{v}"

def fmtInsertVals(v):
    return "" if(v=="") else f",@{v}"

def CharN(v, n):
    tmp = '{:<'+ str(n) +"}"
    return tmp.format(v)


def fmtStrQuote(v):
    # print(v, type(v))
    # return str(v) if(type(v)==int or type(v) == float) else f"'{str(v)}'"
    return str(v) if(str(v).isnumeric()) else f"'{str(v)}'"

def fmtCmmt_ParasInput(r):
    col, AtCol, typLen, ColDscr = r["Column"], r["AtCol"], r["typLen"], r["ColDscr"]
    # typLen = "VARCHAR(10)" if (col == "IsEnable") else typLen
    AtCol = "{:<16}".format(AtCol)
    typLen   = "{:<12}".format(typLen)
    return f"{AtCol} {typLen} - {ColDscr}"

def fmtCmmt_ParasInputQuery(r):
    col, AtCol, typLen, ColDscr = r["Column"], r["AtCol"], r["typLen"], r["ColDscr"]
    
    typLen = "VARCHAR(10)" if (col == "IsEnable") else typLen
    AtCol = "{:<16}".format(AtCol)
    typLen   = "{:<12}".format(typLen)
    return f"{AtCol} {typLen} - {ColDscr}"

def fmtCmmt_returnSelect(r):
    col, typLen, colDscr = r["Column"], r["typLen"], r["ColDscr"]
    col = "{:<16}".format(col)
    typLen   = "{:<12}".format(typLen)
    return f"** {col} {typLen} - {colDscr}"

def fmtTableCompos(r):
    col, typLenTbOnly, autoGen, isNull, colDscr = r["Column"], r["typLenTbOnly"], r["autoGen"], r["isNull"], r["ColDscr"]
    isNull = "NULL" if(isNull == "V") else "NOT NULL"
    autoGen = "IDENTITY(1,1)" if (autoGen =="IDENTITY(1,1)") else ""
    return f"\t[{col}] {typLenTbOnly} {autoGen} {isNull},\t/*{colDscr}*/"

def fmtTableExt(r):
    schema, tableName, col, colDscr, colDscrMemo =  r["Schema"], r["TableName"], r["Column"], r["ColDscr"], r["colDscrMemo"]
    colDscrMemo = "" if (colDscrMemo =="") else f" : {colDscrMemo}"
    colDscr = colDscr + colDscrMemo
    return f"EXEC sys.sp_addextendedproperty @name=N'MS_Description', @value=N'{colDscr}' ,@level0type=N'SCHEMA',@level0name=N'{schema}', @level1type=N'TABLE',@level1name=N'tb{tableName}', @level2type=N'COLUMN',@level2name=N'{col}'\nGO"

def fmtCmmDeclares(r):
    atCol, typLen = r["AtCol"], r["typLen"]
    return f",{atCol} {typLen}"

def fmtCmmDeclaresQuery(r):
    col, atCol, typLen = r["Column"], r["AtCol"], r["typLen"]
    #typLen = "VARCHAR(10)" if(col == "IsEnable") else typLen
    return f",{atCol} {typLen}"

def fmtSetValue(r):
    AtCol, v, typ = r["AtCol"], r["spDelete"], r["Type"]
    v = int(v) if(typ in ["int","decimal","float","tinyint","smallint","numeric"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtExistSetValue(r):
    AtCol, v, typ, funcTyp = r["AtCol"], r["spExist"], r["Type"], r["funcTyp"]
    # print(r["TableName"], r["Column"], typ, v)
    v = int(v) if(typ in ["int","decimal","float","tinyint","smallint","numeric"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtGetSetValue(r):
    AtCol, v, typ, funcTyp = r["AtCol"], r["spGet"], r["Type"], r["funcTyp"]
    # print(r["TableName"], r["Column"], typ, v)
    v = int(v) if(typ in ["int","decimal","float","tinyint","smallint","numeric"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtInsertSetValue(r):
    tmp, AtCol, v, typ, funcTyp = "", r["AtCol"], r["spInsert"], r["Type"], r["funcTyp"]
    # print(AtCol, funcTyp, typ, v)
    if (funcTyp == "V"):
        tmp = f"SET {AtCol} = {v}"
    else:
        v = v if ((typ in ["bit","int","decimal","float","tinyint","smallint","numeric"]) and v != "") else f"'{v}'"
        tmp = f"SET {AtCol} = {v}"
        
    return tmp

def fmtUpdateSetValue(r):
    tmp, AtCol, v, typ, funcTyp = "", r["AtCol"], r["spUpdate"], r["Type"], r["funcTyp"]
    if (funcTyp == "V"):
        tmp = f"SET {AtCol} = {v}"
    else:
        v = v if (typ in ["int","decimal","float","tinyint","smallint","numeric"] and v != "") else f"'{v}'"
        tmp = f"SET {AtCol} = {v}"
    return tmp

def fmtQuerySetValue(r):
    AtCol, v, typ = r["AtCol"], r["spQueryWhere"], r["Type"]
    # print(r["TableName"], r["Column"], typ, v)
    v = int(v) if (typ in ["int","decimal","float","tinyint","smallint","numeric"] and v != "") else f"'{v}'"
    return f"SET {AtCol} = {v}"

def fmtQueryWhere(r):
    tmp, col, dotnetTyp = "", r["Column"], r["dotnetType"]    
    if(dotnetTyp =="int"):
        tmp = f"AND f.$col = @$col"
    elif (dotnetTyp =="string"):
        tmp = f"AND (@$col IS NULL OR f.$col = @$col)" 
    elif (dotnetTyp =="bool"):
        tmp = f"AND (@$col IS NULL OR f.$col = @$col)"   

    tmp = Template(tmp)
    return tmp.substitute(col = col) 

def fmtDotNetType(r):
    t, typ = r["Type"], ""
    if t in ["int","tinyint","smallint","numeric", "float","decimal"]:
        typ = "int"
    elif t in ["varchar","nvarchar","text"]:
        typ = "string"
    elif t in ["date","datetime","datetime2","time"]:
        typ = "DateTime"
    elif t == "bit":
        typ = "bool"
    return typ


def fmtNonCluster(r):
    col, schema, tableName = r["Column"], r["Schema"], r["TableName"]

    tmp = f"""CREATE NONCLUSTERED INDEX [NonClusteredIndex-{col}] ON [{schema}].[{tableName}]
(
	[{col}] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO"""
    tmp = Template(tmp)
    return tmp.substitute(col = col, schema = schema, tableName = tableName) 

def fmtUsingService(r):
    tmp, TableName, SpDscr, tableName, table_name = "", r["TableName"], r["SpDscr"], r["tableName"], r["table_name"]
    return f"using ESUN.AGD.WebApi.Application.{TableName};"

def fmtBuilderService(r):
    tmp, TableName, SpDscr, tableName, table_name = "", r["TableName"], r["SpDscr"], r["tableName"], r["table_name"]
    return f"builder.Services.AddTransient<I{TableName}Service, {TableName}Service>();"

def fmtResponseHandlerCreateMap(r):
    tmp, TableName, SpDscr, tableName, table_name = "", r["TableName"], r["SpDscr"], r["tableName"], r["table_name"]
    return f"CreateMap<Tb{TableName}TransferLog, {TableName}TransferLogResponse>();"

def fmtServiceJsonName(r):
    tmp, TableName, tbDscr = "", r["TableName"], r["tbDscr"]
    TABLENAME = TableName.upper()
    return f''' {{
   "Key": "{TABLENAME}",
   "Name": "{tbDscr}"
 }},'''


In [12]:
## 通用函數：提供 source template file， target: 套表完成的路徑。
def genOneCode(source, target, _dictParas):
    _source = "template/" + source
    fileName = target
    print("source:", _source)
    print("target:", fileName)

    tmp = codecs.open(_source, mode="r",encoding="utf-8", buffering=-1).read()
    t = Template(tmp).substitute(_dictParas)
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t)
    f.close()

import pandas as pd
import numpy as np
import pyodbc
import mycomm as cmm
import time
import os
import codecs
from string import Template

# class MyTemplate(Template):
#     delimiter = "//$"           #自己定義的替換字元符號

#指定Excel表路徑名稱。
_ExcelFile = "refFiles/玉山AGD專案DB_Schema_留言板_20240205M.xlsx"
pd.options.mode.chained_assignment = None  # default='warn'

print(_ExcelFile)
_sheet_name = "DB Schema_留言板新table"
df_table = pd.read_excel(_ExcelFile, sheet_name="Tables",converters={'maxLen':str }).replace(np.nan, '')
# df = pd.read_excel(_ExcelFile, sheet_name = _sheet_name ,converters={'maxLen':str }).replace(np.nan, '')


#### 欄位轉型 FieldTransform ------------------------------------------------------------------------------------
df_table["usingService"]                = df_table.apply(fmtUsingService, axis = 1)
df_table["builderService"]              = df_table.apply(fmtBuilderService, axis = 1)
df_table["serviceJson"]                 = df_table.apply(fmtServiceJsonName, axis = 1)
df_table["responseHandlerCreateMap"]    = df_table.apply(fmtResponseHandlerCreateMap, axis = 1)

#### 欄位組合 Filed Join  ------------------------------------------------------------------------------------
tbs = df_table[df_table["isCopy"]=="V"] #挑選主要的Table
pt_UsingService             = "\n".join(tbs["usingService"].tolist())
pt_BuilderService           = "\n".join(tbs["builderService"].tolist())
pt_ServiceJson              = "\n".join(tbs["serviceJson"].tolist())
pt_ResponseHandlerCreateMap = "\n".join(tbs["responseHandlerCreateMap"].tolist())

print(tbs)
dictParas = dict(pt_UsingService = pt_UsingService, pt_BuilderService = pt_BuilderService, pt_ServiceJson = pt_ServiceJson, pt_ResponseHandlerCreateMap = pt_ResponseHandlerCreateMap)
## 開始套表...
genOneCode("WebApi/Program.cs",                                 f"WebApi/Program.cs", dictParas)
genOneCode("WebApi/Service.json",                               f"WebApi/Serice.json", dictParas)
genOneCode("WebApi/Application/Common/ResponseHandler.cs",      f"WebApi/Application/Common/ResponseHandler.cs", dictParas)


refFiles/玉山AGD專案DB_Schema_留言板_20240205M.xlsx
        tbDscr                     TableName SpDscr  \
0       訪客留言紀錄                        WebMsg          
1     聯繫紀錄明細暫存  ContactCustomerReasonStaging          
2     訪客留言移轉紀錄             WebMsgTransferLog          
3     訪客留言審核紀錄               WebMsgReviewLog          
4       留言類型處理                       MsgItem          
5  留言Email回覆範本              MsgEmailTemplate          

                      tableName                       table_name isCopy  \
0                        webMsg                          web-msg      V   
1  contactCustomerReasonStaging  contact-customer-reason-staging      V   
2             webMsgTransferLog             web-msg-transfer-log      V   
3               webMsgReviewLog               web-msg-review-log      V   
4                       msgItem                         msg-item      V   
5              msgEmailTemplate               msg-email-template      V   

                                        usi